### Importing libraries and reading data

In [45]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')

In [46]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)

### Parsing html

In [47]:
tableSoup = soup.find_all("table")
table = tableSoup[0]
rows = table.find_all("tr")
for row in rows:
    columns = row.find_all("td")
    headers = row.find_all("th")
    if len(columns) == 0 : continue
    postcode = columns[0].text
    borough = columns[1].text
    neigh = columns[2].text.split("\n")[0]
    neighborhoods = neighborhoods.append({'PostalCode': postcode,
                          'Borough': borough,
                          'Neighborhood': neigh}, ignore_index=True)
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


###  Ignore cells with a borough that is Not assigned.

In [48]:
neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains("[^Not assigned]")==True]
neighborhoods.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Combining neighborhoods with the same Postal Code into one row with the neighborhoods separated with a comma

In [49]:
neighborhoods = pd.DataFrame({'Neighborhood' : neighborhoods.groupby(by = ['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))}).reset_index()
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Neighborhood not assigned, then the neighborhood will be the same as the borough

In [50]:
neighborhoods['Neighborhood'].replace(to_replace="Not assigned", value=neighborhoods['Borough'], inplace=True)
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### DataFrame shape

In [53]:
neighborhoods.shape

(103, 3)